In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys

# WebDriver başlatma
driver = uc.Chrome()

try:
    # Siteye git
    driver.get("https://sim.csb.gov.tr/STN/STN_Report/StationDataDownloadNew")
    time.sleep(4)


    # Ankara seçeneğini seçme
    try:

        # Input alanını bulma ve yazma işlemi
        input_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="dropdown1-contentDataDowloadNew"]/div/div/div/div/input'))
        )
        input_element.send_keys("Ankara")  # "Ankara" yaz
        time.sleep(1)  # Güncellenmesini bekle
        input_element.send_keys(Keys.ENTER)  # Enter tuşuna basarak seçimi tamamla

         # İl seçimi sonrası herhangi bir yere tıklama
        body_element = driver.find_element(By.TAG_NAME, 'body')  # Sayfanın herhangi bir yerine tıklama
        body_element.click()
        time.sleep(2)

    except Exception as e:
        print("Ankara seçimi yapılamadı.", e)

    gunluk_secimi = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="dropdown4-contentDataDowloadNew"]/div/div/label[2]'))
        )
    gunluk_secimi.click()
    time.sleep(1)



    parametre_secimi = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="dropdown3-contentDataDowloadNew"]/div/div/div/div/input'))
        )
    parametre_secimi.click()   
    time.sleep(1)

    parametre_secimi.send_keys(Keys.ENTER)  # Enter tuşuna basarak seçimi tamamla

    # Dinamik olarak tüm seçenekleri seçme
    for i in range(6):  # `len(options)` ile dinamik olarak seçenek sayısını alıyoruz
        parametre_secimi.send_keys(Keys.ARROW_DOWN)  # Bir sonraki seçeneğe geç
        time.sleep(0.5)  # Kısa bekleme
        parametre_secimi.send_keys(Keys.ENTER)  # Seçimi yap
        time.sleep(0.5)  # Kısa bekleme

     # Parametre seçimi sonrası herhangi bir yere tıklama
    body_element = driver.find_element(By.TAG_NAME, 'body')  # Sayfanın herhangi bir yerine tıklama
    body_element.click()
    time.sleep(2)    


    # İlgili input alanını bekle ve seç
    date_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="StationDataDownload_StartDateTime"]'))
    )

    # İçindeki metni temizle
    date_input.clear()

    # Yeni tarihi yaz
    date_input.send_keys('21.11.2023')
        
    # Tarih seçimi sonrası herhangi bir yere tıklama
    body_element = driver.find_element(By.TAG_NAME, 'body')  # Sayfanın herhangi bir yerine tıklama
    body_element.click()
    time.sleep(2)   



    # Boş bir DataFrame oluştur
    df = pd.DataFrame(columns=["Column1", "Column2", "Column3", "Column4", "Column5", "Column6", "Column7"])
    
    # İlçeleri sırayla seçmek için döngü
    current_index = 0  # Mevcut ilçeyi izlemek için sayaç
    max_ilce_count = 18  # İlçe sayısı sınırı


    while True:
        try:
            
            # İlçe dropdown'ını aç
            ilce_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="dropdown2-contentDataDowloadNew"]/div/div/span[1]'))
            )
            ilce_dropdown.click()
            time.sleep(1)
            # Açılan dropdown'daki input alanına odaklan ve tıkla
            ilce_secimi = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.k-list-container span input.k-textbox'))
            )
            ilce_secimi.click()
            time.sleep(1)

            # İlçe seçimini yap
            ilce_secimi.send_keys(Keys.ARROW_DOWN)  # Bir sonraki seçeneğe geç
            time.sleep(0.5)
            ilce_secimi.send_keys(Keys.ENTER)  # Seçimi yap
            time.sleep(1)

            # İlçe seçimi sonrası herhangi bir yere tıklama
            body_element = driver.find_element(By.TAG_NAME, 'body')  # Sayfanın herhangi bir yerine tıklama
            body_element.click()
            time.sleep(2)

            # Arama butonuna tıklama
            try:
                search_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="StationDataDownloadForm"]/fieldset[1]/div[1]/div[2]/div[1]/div/div/div/button'))
                )
                search_button.click()
                time.sleep(3)
            except Exception as e:
                print("Arama butonuna tıklanamadı.", e)

            # Satır sayısının seçilmesi
            dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.k-pager-sizes.k-label > span.k-widget.k-dropdown.k-header'))
            )
            dropdown.click()
            time.sleep(1)

            # Görünür olan ul içerisindeki belirli bir li'yi seç
            option = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.k-animation-container[style*="display: block"] ul.k-list > li[data-offset-index="4"]'))
            )
            option.click()


            # Tabloyu kesin olarak seçmek için detaylı seçici kullan
            table_selector = 'table[role="grid"]'

            # Tablo başlıklarını bul
            column_headers = [
                th.get_attribute('data-title').strip()
                for th in driver.find_elements(By.CSS_SELECTOR, f'{table_selector} thead tr:nth-child(2) th')
            ]
            print("Sütun Başlıkları:", column_headers)

            if df.empty:
                df = pd.DataFrame(columns=column_headers)
            
            # Her sütun için verileri doğrudan al
            column1 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(1)')]
            column2 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(2)')]
            column3 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(3)')]
            column4 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(4)')]
            column5 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(5)')]
            column6 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(6)')]
            column7 = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'table[role="grid"] tbody tr td:nth-child(7)')]

            # Verilerin boyutlarını kontrol et
            print("Veri boyutları:", len(column1), len(column2), len(column3), len(column4), len(column5), len(column6), len(column7))

            # Boyutlar eşit değilse, minimum uzunluk kadarını al
            min_length = min(len(column1), len(column2), len(column3), len(column4), len(column5), len(column6), len(column7))
            column1 = column1[:min_length]
            column2 = column2[:min_length]
            column3 = column3[:min_length]
            column4 = column4[:min_length]
            column5 = column5[:min_length]
            column6 = column6[:min_length]
            column7 = column7[:min_length]
            
        # Sütun başlıklarını kontrol et
            if len(column_headers) == 7:
                temp_df = pd.DataFrame({
                    column_headers[0]: column1,
                    column_headers[1]: column2,
                    column_headers[2]: column3,
                    column_headers[3]: column4,
                    column_headers[4]: column5,
                    column_headers[5]: column6,
                    column_headers[6]: column7,
                })
            else:
                 # İlçe sayısına ulaşıldığında döngüyü sonlandır
                if current_index == max_ilce_count:
                    print("Tüm ilçeler işleme alındı.")
                    break
                print(f"Sütun sayısı uyuşmadı. Beklenen: 7, Bulunan: {len(column_headers)}. İlçe atlanıyor...")
                current_index += 1  # İlçe sayacını artır
                continue  # Sonraki ilçeye geç
            
            # `temp_df` sütunlarını `df` ile uyumlu hale getir
            temp_df = temp_df.reindex(columns=df.columns)

            # Ana DataFrame ile birleştir
            df = pd.concat([df, temp_df], ignore_index=True)
            print(f"DataFrame'e yeni veriler eklendi. Şu anki toplam satır sayısı: {len(df)}")

            # Veri eklendiğini bildiren mesaj
            print(f"İlçe {current_index + 1}/{max_ilce_count} tamamlandı. Toplam satır sayısı: {len(df)}")
          
            # Mevcut indexi artır
            current_index += 1

            # İlçe sayısına ulaşıldığında döngüyü sonlandır
            if current_index == max_ilce_count:
                print("Tüm ilçeler işleme alındı.")
                break



        except Exception as e:
            # Eğer tüm ilçeler seçilmişse veya bir hata alırsak döngüyü kır
            print("Tüm ilçeler seçildi veya hata oluştu:", e)
            break   
                

    # DataFrame'i yazdır veya kaydet
    print("Tüm veriler toplandı.")
    print(df)

finally:
    # Tarayıcıyı kapatma
    driver.quit()


Sütun Başlıkları: ['PM10 ( µg/m3 )', 'PM 2.5 ( µg/m3 )', 'SO2 ( µg/m3 )', 'CO ( µg/m3 )', 'NO2 ( µg/m3 )', 'NOX ( µg/m3 )', 'NO ( µg/m3 )']
Veri boyutları: 378 377 377 377 377 377 377
DataFrame'e yeni veriler eklendi. Şu anki toplam satır sayısı: 377
İlçe 1/18 tamamlandı. Toplam satır sayısı: 377
Sütun Başlıkları: ['PM10 ( µg/m3 )', 'PM 2.5 ( µg/m3 )', 'SO2 ( µg/m3 )', 'CO ( µg/m3 )', 'NO2 ( µg/m3 )', 'NOX ( µg/m3 )', 'NO ( µg/m3 )']
Veri boyutları: 378 377 377 377 377 377 377
DataFrame'e yeni veriler eklendi. Şu anki toplam satır sayısı: 754
İlçe 2/18 tamamlandı. Toplam satır sayısı: 754
Sütun Başlıkları: ['PM10 ( µg/m3 )', 'PM 2.5 ( µg/m3 )', 'CO ( µg/m3 )', 'NO2 ( µg/m3 )', 'NOX ( µg/m3 )', 'NO ( µg/m3 )']
Veri boyutları: 378 377 377 377 377 377 8
Sütun sayısı uyuşmadı. Beklenen: 7, Bulunan: 6. İlçe atlanıyor...
Sütun Başlıkları: ['PM10 ( µg/m3 )', 'PM 2.5 ( µg/m3 )', 'SO2 ( µg/m3 )', 'NO2 ( µg/m3 )', 'NOX ( µg/m3 )', 'NO ( µg/m3 )']
Veri boyutları: 378 377 377 377 377 377 8
Sütun s

In [65]:
df[:-5]

,PM10 ( µg/m3 ),PM 2.5 ( µg/m3 ),SO2 ( µg/m3 ),CO ( µg/m3 ),NO2 ( µg/m3 ),NOX ( µg/m3 ),NO ( µg/m3 )
0,"21,91","9,36","4,07",-,"71,92","115,55","43,63"
1,"51,51","39,55","4,19",-,"115,10","198,74","83,64"
2,"52,74","40,67","3,94",-,"127,38","241,35","113,97"
3,"49,38","25,98","3,27",-,"84,65","143,17","58,52"
4,"62,63","20,04","2,48",-,"32,07","37,06","4,99"
...,...,...,...,...,...,...,...
2252,"12,87",-,"0,81","271,14","31,56","43,30","11,74"
2253,-,-,-,-,-,-,-
2254,İstasyon: Ankara - Polatlı,Ankara - Polatlı,PM10,µg/m3,"3,25",24.12.2023 00:00,"155,23"
2255,,Ankara - Polatlı,PM 2.5,µg/m3,,,
